In [1]:
import re
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

# Const

In [2]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'Rare_Earth/'
dir_tableau = dir_proj + 'tableau_data/'

rares = {}

years = range(2002, 2017)
files = [f'rareemyb{str(y)[-2:]}.xlsm' if y < 2006 else f'myb1-{y}-raree.xlsm' if y < 2016 else f'myb1-{y}-raree-adv.xlsm' for y in years]
paths = dict(zip(years, [dir_proj + f for f in files]))
paths

{2002: '/home/jake/Documents/News_Item/Rare_Earth/rareemyb02.xlsm',
 2003: '/home/jake/Documents/News_Item/Rare_Earth/rareemyb03.xlsm',
 2004: '/home/jake/Documents/News_Item/Rare_Earth/rareemyb04.xlsm',
 2005: '/home/jake/Documents/News_Item/Rare_Earth/rareemyb05.xlsm',
 2006: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2006-raree.xlsm',
 2007: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2007-raree.xlsm',
 2008: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2008-raree.xlsm',
 2009: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2009-raree.xlsm',
 2010: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2010-raree.xlsm',
 2011: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2011-raree.xlsm',
 2012: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2012-raree.xlsm',
 2013: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2013-raree.xlsm',
 2014: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2014-raree.xlsm',
 2015: '/home/jake/Documents/News_Item/Rare_Earth/myb1-2015-raree.xlsm'

# Load

In [3]:
def load_and_prepare_import(path, sheet_name=5, header=5):
    if ('myb03' in path) or ('myb02' in path):
        sheet_name = 4
    tmp = pd.read_excel(path, sheet_name, header)
    #     tmp = tmp.iloc[::, :10]
    tmp['does_number_exist'] = tmp.iloc[::, 2:].any(axis=1)
    tmp['does_indent_exist'] = ~tmp.iloc[::, 1].isna()
    tmp['is_country'] = tmp['does_number_exist'] & tmp['does_indent_exist']
    tmp['is_category'] = ~tmp['does_number_exist'] & tmp['does_indent_exist']
    tmp['is_note'] = ~tmp['does_number_exist'] & ~tmp['does_indent_exist']
    return tmp
    
# tmp = load_and_prepare_import(path_2016)
# tmp.head()

# Notes

In [4]:
# notes = tmp.loc[tmp['is_note'], tmp.columns[0]]; notes

# get_rare

In [5]:
def get_rare(tmp, recorded_year, max_indent=7):
    collection = []
    current_year = recorded_year
    previous_year = recorded_year - 1
    current_category = [''] * max_indent
    
    for idx, row in tmp.iterrows():
        if row['is_note']:
            continue
        indent = int(row[1])
        if row['is_category']:
            current_category[indent] = row[0]
        elif row['is_country']:
            if 'Total' in row[0]:
                start = indent - 1
            else:
                start = indent
            for c in reversed(current_category[:start]):
                obj = re.search(r'\(([\d\.,\s\*]+)\)', c)
                if obj:
                    code = obj.group(1)
                    break
                else:
                    code = ''
            previous_year_data = [code, previous_year, recorded_year, row[0]]  + row[2:6].to_list()
            current_year_data = [code, current_year, recorded_year, row[0]] + row[6:10].to_list()
            collection.append(previous_year_data)
            collection.append(current_year_data)
                                
    rare = pd.DataFrame(columns=['hts_code', 'year', 'recorded_year', 'country', 'weight(kilogram)', 'weight_revised', 'value(US dollar)', 'value_revised'], data=collection)
    return rare
                        
                        
# rare_2016 = get_rare(tmp)\n# rare_2016

# get_rare_or_tmp

In [6]:
def get_rare_or_tmp(year, debug=False):
    tmp = load_and_prepare_import(paths[year])
    if debug:
        return tmp
    
    return get_rare(tmp, year)

In [7]:
year = 2016
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0010,2015,2016,China,402000,NaN,1770000,NaN
1,2846.10.0010,2016,2016,China,801000,NaN,1770000,NaN
2,2846.10.0010,2015,2016,Japan,292000,NaN,13500000,NaN
3,2846.10.0010,2016,2016,Japan,332000,NaN,13100000,NaN
4,2846.10.0010,2015,2016,Other,35000,NaN,656000,NaN
5,2846.10.0010,2016,2016,Other,18700,NaN,723000,NaN
6,2846.10.0010,2015,2016,Total,730000,NaN,15900000,NaN
7,2846.10.0010,2016,2016,Total,1150000,NaN,15600000,NaN
8,2846.10.0010,2015,2016,Total estimated rare-earth-oxide (REO) equivalent,730000,NaN,XX,NaN
9,2846.10.0010,2016,2016,Total estimated rare-earth-oxide (REO) equivalent,1150000,NaN,XX,NaN


In [8]:
year = 2015
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0010,2014,2015,China,307000,NaN,1680000,NaN
1,2846.10.0010,2015,2015,China,402000,NaN,1770000,True
2,2846.10.0010,2014,2015,Japan,309000,NaN,12300000,NaN
3,2846.10.0010,2015,2015,Japan,292000,NaN,13500000,True
4,2846.10.0010,2014,2015,Other,73600,r,897000,r
5,2846.10.0010,2015,2015,Other,35000,NaN,656000,True
6,2846.10.0010,2014,2015,Total,690000,NaN,14900000,NaN
7,2846.10.0010,2015,2015,Total,730000,NaN,15900000,True
8,2846.10.0010,2014,2015,Total estimated equivalent rare-earth oxide (REO) content,690000,NaN,XX,NaN
9,2846.10.0010,2015,2015,Total estimated equivalent rare-earth oxide (REO) content,730000,NaN,XX,True


In [9]:
year = 2014
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2013,2014,Austria,73400,NaN,1780000,NaN
1,2846.10.0000,2014,2014,Austria,XX,NaN,XX,True
2,2846.10.0000,2013,2014,China,360000,NaN,2890000,NaN
3,2846.10.0000,2014,2014,China,XX,NaN,XX,True
4,2846.10.0000,2013,2014,Estonia,30200,NaN,416000,
5,2846.10.0000,2014,2014,Estonia,XX,NaN,XX,True
6,2846.10.0000,2013,2014,Japan,77200,NaN,3580000,NaN
7,2846.10.0000,2014,2014,Japan,XX,NaN,XX,True
8,2846.10.0000,2013,2014,Other,57700,NaN,812000,NaN
9,2846.10.0000,2014,2014,Other,XX,NaN,XX,True


In [10]:
year = 2013
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2012,2013,Austria,187000,NaN,8030000,NaN
1,2846.10.0000,2013,2013,Austria,73400,NaN,1780000,True
2,2846.10.0000,2012,2013,China,1040000,NaN,24200000,NaN
3,2846.10.0000,2013,2013,China,360000,NaN,2890000,True
4,2846.10.0000,2012,2013,Estonia,428000,NaN,6020000,
5,2846.10.0000,2013,2013,Estonia,30200,NaN,416000,True
6,2846.10.0000,2012,2013,Japan,189000,NaN,10300000,NaN
7,2846.10.0000,2013,2013,Japan,77200,NaN,3580000,True
8,2846.10.0000,2012,2013,Spain,80300,NaN,816000,
9,2846.10.0000,2013,2013,Spain,--,NaN,--,True


In [11]:
year = 2012
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2011,2012,Austria,302000,NaN,10400000,NaN
1,2846.10.0000,2012,2012,Austria,187000,NaN,8030000,True
2,2846.10.0000,2011,2012,China,640000,NaN,38500000,NaN
3,2846.10.0000,2012,2012,China,1040000,NaN,24200000,True
4,2846.10.0000,2011,2012,Estonia,182000,NaN,10600000,
5,2846.10.0000,2012,2012,Estonia,428000,NaN,6020000,True
6,2846.10.0000,2011,2012,India,192000,,7370000,
7,2846.10.0000,2012,2012,India,150,NaN,17200,True
8,2846.10.0000,2011,2012,Japan,168000,NaN,8710000,NaN
9,2846.10.0000,2012,2012,Japan,189000,NaN,10300000,True


In [12]:
year = 2011
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2010,2011,Austria,131000,NaN,1400000,NaN
1,2846.10.0000,2011,2011,Austria,302000,NaN,10400000,NaN
2,2846.10.0000,2010,2011,China,2010000,NaN,13500000,NaN
3,2846.10.0000,2011,2011,China,640000,NaN,38500000,NaN
4,2846.10.0000,2010,2011,Estonia,200000,,1040000,
5,2846.10.0000,2011,2011,Estonia,182000,NaN,10600000,NaN
6,2846.10.0000,2010,2011,France,11800,NaN,685000,NaN
7,2846.10.0000,2011,2011,France,48000,NaN,2370000,NaN
8,2846.10.0000,2010,2011,Japan,139000,NaN,4700000,NaN
9,2846.10.0000,2011,2011,Japan,168000,NaN,8710000,NaN


In [13]:
year = 2010
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2009,2010,Austria,386000,NaN,1140000,NaN
1,2846.10.0000,2010,2010,Austria,131000,NaN,1400000,NaN
2,2846.10.0000,2009,2010,China,1720000,NaN,5030000,NaN
3,2846.10.0000,2010,2010,China,2010000,NaN,13500000,NaN
4,2846.10.0000,2009,2010,France,17700,NaN,208000,NaN
5,2846.10.0000,2010,2010,France,11800,NaN,685000,NaN
6,2846.10.0000,2009,2010,Japan,51600,NaN,1800000,NaN
7,2846.10.0000,2010,2010,Japan,139000,NaN,4700000,NaN
8,2846.10.0000,2009,2010,"Korea, Republic of",--,NaN,--,NaN
9,2846.10.0000,2010,2010,"Korea, Republic of",32400,NaN,275000,NaN


In [14]:
year = 2009
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2008,2009,Austria,78700,NaN,734000,NaN
1,2846.10.0000,2009,2009,Austria,386000,NaN,1140000,True
2,2846.10.0000,2008,2009,China,2890000,NaN,9980000,NaN
3,2846.10.0000,2009,2009,China,1720000,NaN,5030000,True
4,2846.10.0000,2008,2009,France,36800,NaN,421000,NaN
5,2846.10.0000,2009,2009,France,17700,NaN,208000,True
6,2846.10.0000,2008,2009,Japan,43600,NaN,1440000,NaN
7,2846.10.0000,2009,2009,Japan,51600,NaN,1800000,True
8,2846.10.0000,2008,2009,"Korea, Republic of",34,NaN,21600,NaN
9,2846.10.0000,2009,2009,"Korea, Republic of",--,NaN,--,True


In [15]:
year = 2008
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2007,2008,Austria,60600,NaN,356000,NaN
1,2846.10.0000,2008,2008,Austria,78700,NaN,734000,True
2,2846.10.0000,2007,2008,China,3630000,NaN,7770000,NaN
3,2846.10.0000,2008,2008,China,2890000,NaN,9980000,True
4,2846.10.0000,2007,2008,France,121000,NaN,1080000,NaN
5,2846.10.0000,2008,2008,France,36800,NaN,421000,True
6,2846.10.0000,2007,2008,Hong Kong,16500,NaN,20600,NaN
7,2846.10.0000,2008,2008,Hong Kong,--,NaN,--,True
8,2846.10.0000,2007,2008,Japan,88700,NaN,2350000,NaN
9,2846.10.0000,2008,2008,Japan,43600,NaN,1440000,True


In [16]:
year = 2007
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2006,2007,Austria,36400,NaN,235000,NaN
1,2846.10.0000,2007,2007,Austria,60600,NaN,356000,True
2,2846.10.0000,2006,2007,China,3540000,NaN,6350000,NaN
3,2846.10.0000,2007,2007,China,3630000,NaN,7770000,True
4,2846.10.0000,2006,2007,France,73600,NaN,884000,NaN
5,2846.10.0000,2007,2007,France,121000,NaN,1080000,True
6,2846.10.0000,2006,2007,Hong Kong,64000,NaN,127000,NaN
7,2846.10.0000,2007,2007,Hong Kong,16500,NaN,20600,True
8,2846.10.0000,2006,2007,Japan,135000,NaN,3090000,NaN
9,2846.10.0000,2007,2007,Japan,88700,NaN,2350000,True


In [17]:
year = 2006
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2005,2006,Austria,18300,NaN,188000,NaN
1,2846.10.0000,2006,2006,Austria,36400,NaN,235000,True
2,2846.10.0000,2005,2006,China,2.84e+06,NaN,4.89e+06,NaN
3,2846.10.0000,2006,2006,China,3540000,NaN,6350000,True
4,2846.10.0000,2005,2006,France,83700,NaN,1e+06,NaN
5,2846.10.0000,2006,2006,France,73600,NaN,884000,True
6,2846.10.0000,2005,2006,Hong Kong,36000,NaN,77500,NaN
7,2846.10.0000,2006,2006,Hong Kong,64000,NaN,127000,True
8,2846.10.0000,2005,2006,Ireland,13900,NaN,350000,NaN
9,2846.10.0000,2006,2006,Ireland,--,NaN,--,True


In [18]:
year = 2005
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2004,2005,Austria,39000,NaN,371000,NaN
1,2846.10.0000,2005,2005,Austria,18300,NaN,188000,True
2,2846.10.0000,2004,2005,China,2260000,NaN,4450000,NaN
3,2846.10.0000,2005,2005,China,2840000,NaN,4890000,True
4,2846.10.0000,2004,2005,France,190000,NaN,2500000,NaN
5,2846.10.0000,2005,2005,France,83700,NaN,1000000,True
6,2846.10.0000,2004,2005,Hong Kong,--,NaN,--,NaN
7,2846.10.0000,2005,2005,Hong Kong,36000,NaN,77500,True
8,2846.10.0000,2004,2005,Ireland,13900,NaN,350000,NaN
9,2846.10.0000,2005,2005,Ireland,13900,NaN,350000,True


In [19]:
year = 2004
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2003,2004,Austria,166000,NaN,677000,NaN
1,2846.10.0000,2004,2004,Austria,39000,NaN,371000,True
2,2846.10.0000,2003,2004,China,3050000,NaN,7170000,NaN
3,2846.10.0000,2004,2004,China,2260000,NaN,4450000,True
4,2846.10.0000,2003,2004,France,155000,NaN,2390000,NaN
5,2846.10.0000,2004,2004,France,190000,NaN,2500000,True
6,2846.10.0000,2003,2004,Japan,174000,r,4290000,NaN
7,2846.10.0000,2004,2004,Japan,270000,NaN,5050000,True
8,2846.10.0000,2003,2004,Other,80300,NaN,533000,NaN
9,2846.10.0000,2004,2004,Other,56600,NaN,777000,True


In [20]:
year = 2003
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2002,2003,Austria,76500,NaN,337000,NaN
1,2846.10.0000,2003,2003,Austria,166000,NaN,677000,True
2,2846.10.0000,2002,2003,China,2770000,NaN,9050000,NaN
3,2846.10.0000,2003,2003,China,3050000,NaN,7170000,True
4,2846.10.0000,2002,2003,France,725000,NaN,4420000,NaN
5,2846.10.0000,2003,2003,France,155000,NaN,2390000,True
6,2846.10.0000,2002,2003,Japan,162000,NaN,4980000,NaN
7,2846.10.0000,2003,2003,Japan,172000,NaN,4290000,True
8,2846.10.0000,2002,2003,Other,73600,NaN,348000,NaN
9,2846.10.0000,2003,2003,Other,80300,NaN,533000,True


In [21]:
year = 2002
rares[year] = get_rare_or_tmp(year)
rares[year]

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0000,2001,2002,Austria,59000,NaN,439000,NaN
1,2846.10.0000,2002,2002,Austria,76500,NaN,337000,True
2,2846.10.0000,2001,2002,China,4060000,NaN,14000000,NaN
3,2846.10.0000,2002,2002,China,2770000,NaN,9050000,True
4,2846.10.0000,2001,2002,France,1240000,NaN,6650000,NaN
5,2846.10.0000,2002,2002,France,725000,NaN,4420000,True
6,2846.10.0000,2001,2002,Japan,288000,NaN,6500000,NaN
7,2846.10.0000,2002,2002,Japan,162000,NaN,4980000,True
8,2846.10.0000,2001,2002,Other,115000,NaN,737000,NaN
9,2846.10.0000,2002,2002,Other,73600,NaN,348000,True


# Merge

In [22]:
# tmp_rare = rare_2016.append(rare_2015)
# tmp_rare = tmp_rare.append(rare_2014)
# tmp_rare = tmp_rare.append(rare_2013)
# tmp_rare = tmp_rare.append(rare_2012)
# tmp_rare.head()
tmp_rare = pd.DataFrame()
for k, v in rares.items():
    tmp_rare = tmp_rare.append(v)
tmp_rare.shape

(2288, 8)

# Prepare

In [23]:
import numpy as np
rare = tmp_rare.loc[(tmp_rare['recorded_year']==2016) | (tmp_rare['recorded_year'] > tmp_rare['year'])]
rare.replace('--', 0, inplace=True)
rare.replace('XX', np.nan, inplace=True)
rare['country'] = rare['country'].apply(lambda x: x.strip())
# rare.applymap(lambda x: x.strip())
rare.reset_index(inplace=True, drop=True)
rare.head()

/home/jake/.virtualenvs/workspace/lib/python3.6/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/home/jake/.virtualenvs/workspace/lib/python3.6/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0010,2015,2016,China,402000.0,NaN,1770000.0,NaN
1,2846.10.0010,2016,2016,China,801000.0,NaN,1770000.0,NaN
2,2846.10.0010,2015,2016,Japan,292000.0,NaN,13500000.0,NaN
3,2846.10.0010,2016,2016,Japan,332000.0,NaN,13100000.0,NaN
4,2846.10.0010,2015,2016,Other,35000.0,NaN,656000.0,NaN


In [24]:
rare['hts_code'].value_counts()

2846.90.8000                                146
2805.30.0000                                122
2846.90.2050                                121
2846.10.0000                                117
2846.90.2010                                111
3606.90.3000                                104
2846.90.4000                                97 
2846.10.0050                                29 
2805.30.0020                                29 
2846.90.8090                                28 
2846.10.0010                                27 
2805.30.0090                                26 
2846.90.2015                                25 
2846.90.2084                                24 
2846.90.8075                                23 
2846.90.8070                                22 
2805.30.0005                                22 
2805.30.0010                                22 
2846.90.2005                                20 
2805.30.0050                                20 
2846.90.2040                            

In [25]:
rare

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0010,2015,2016,China,402000.00,NaN,1770000.0,NaN
1,2846.10.0010,2016,2016,China,801000.00,NaN,1770000.0,NaN
2,2846.10.0010,2015,2016,Japan,292000.00,NaN,13500000.0,NaN
3,2846.10.0010,2016,2016,Japan,332000.00,NaN,13100000.0,NaN
4,2846.10.0010,2015,2016,Other,35000.00,NaN,656000.0,NaN
5,2846.10.0010,2016,2016,Other,18700.00,NaN,723000.0,NaN
6,2846.10.0010,2015,2016,Total,730000.00,NaN,15900000.0,NaN
7,2846.10.0010,2016,2016,Total,1150000.00,NaN,15600000.0,NaN
8,2846.10.0010,2015,2016,Total estimated rare-earth-oxide (REO) equivalent,730000.00,NaN,NaN,NaN
9,2846.10.0010,2016,2016,Total estimated rare-earth-oxide (REO) equivalent,1150000.00,NaN,NaN,NaN


In [26]:
rare.loc[rare['country']=='Total', 'weight(kilogram)'].sum()

295776972.0

# Export hts_codes

In [27]:
# rare[['hts_code', 'year', 'country', 'weight(kilogram)', 'value(US dollar)']].to_csv(dir_tableau + 'hts_codes.tsv', sep='\t', index=False)

In [28]:
mat_0 = rare['hts_code']=='2846.90.8090'
mat_1 = rare['hts_code'] == '2846.90.4000'
mat_2 = rare['hts_code'] == '3606.90.3000'
mat_3 = rare['hts_code'] == '2846.90.8000'
mat_4 = rare['hts_code'] == '2846.90.8000, 2846.90.8090'
china = rare['country'].str.contains('China')

In [29]:
rare.head()

,hts_code,year,recorded_year,country,weight(kilogram),weight_revised,value(US dollar),value_revised
0,2846.10.0010,2015,2016,China,402000.0,NaN,1770000.0,NaN
1,2846.10.0010,2016,2016,China,801000.0,NaN,1770000.0,NaN
2,2846.10.0010,2015,2016,Japan,292000.0,NaN,13500000.0,NaN
3,2846.10.0010,2016,2016,Japan,332000.0,NaN,13100000.0,NaN
4,2846.10.0010,2015,2016,Other,35000.0,NaN,656000.0,NaN


In [30]:
idx_2016 = rare['year']==2016

In [31]:
rare_2016 = rare.loc[idx_2016].pivot(index='hts_code', columns='country', values='weight(kilogram)')
rare_2016.fillna(0, inplace=True)
rare_2016['china_rate'] = rare_2016['China'] / rare_2016['Total'] * 100
print(rare_2016.columns)

rare_2016

Index(['Austria', 'China', 'Estonia', 'France', 'Germany', 'Hong Kong',
       'Japan', 'Korea, Republic of', 'Malaysia', 'Other', 'Spain', 'Thailand',
       'Total', 'Total estimated REO equivalent',
       'Total estimated rare-earth-oxide (REO) equivalent', 'United Kingdom',
       'china_rate'],
      dtype='object', name='country')


country,Austria,China,Estonia,France,Germany,Hong Kong,Japan,"Korea, Republic of",Malaysia,Other,Spain,Thailand,Total,Total estimated REO equivalent,Total estimated rare-earth-oxide (REO) equivalent,United Kingdom,china_rate
hts_code,,,,,,,,,,,,,,,,,
2805.30.0005,0.0,43100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,352.0,0.0,0.0,43400.0,50900.0,0.0,0.0,99.308756
2805.30.0010,0.0,64700.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,301.0,0.0,0.0,85000.0,104000.0,0.0,0.0,76.117647
2805.30.0020,0.0,4060.0,0.0,0.0,0.0,0.0,601.0,0.0,0.0,500.0,0.0,0.0,7170.0,8360.0,0.0,2000.0,56.624826
2805.30.0050,0.0,37500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5950.0,0.0,0.0,43400.0,52100.0,0.0,0.0,86.405530
2805.30.0090,0.0,139000.0,0.0,0.0,0.0,0.0,14100.0,0.0,0.0,3560.0,0.0,0.0,157000.0,188000.0,0.0,0.0,88.535032
2846.10.0010,0.0,801000.0,0.0,0.0,0.0,0.0,332000.0,0.0,0.0,18700.0,0.0,0.0,1150000.0,0.0,1150000.0,0.0,69.652174
2846.10.0050,0.0,424000.0,525000.0,0.0,0.0,0.0,0.0,0.0,0.0,68800.0,0.0,0.0,1020000.0,681000.0,0.0,0.0,41.568627
2846.90.2005,0.0,3320000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11300.0,0.0,0.0,3330000.0,3330000.0,0.0,0.0,99.699700
2846.90.2015,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,3660.0,0.0,1690.0,0.0,0.0,9340.0,9340.0,0.0,0.0,42.826552


In [32]:
summary_2016 = rare_2016.sum()
summary_2016 = summary_2016.to_frame().reset_index().rename(columns={0: 'sum'})
total_2016 = float(summary_2016.loc[summary_2016['country']=='Total', 'sum'])
summary_2016['rate'] = summary_2016['sum'] / total_2016 * 100
summary_2016
# total_2016

,country,sum,rate
0,Austria,5.180000e+04,0.305162
1,China,1.385551e+07,81.625102
2,Estonia,9.460000e+05,5.573042
3,France,7.090000e+05,4.176836
4,Germany,2.200000e+03,0.012961
5,Hong Kong,2.000000e+04,0.117823
6,Japan,3.469830e+05,2.044134
7,"Korea, Republic of",3.769000e+03,0.022204
8,Malaysia,4.400000e+05,2.592113
9,Other,5.484420e+05,3.230963


# Export summary

In [33]:
# summary_2016.to_csv(dir_tableau + '2016_summary.tsv', sep='\t', index=False)

In [34]:
# rare_2016[['China', 'Total', 'china_rate',]].reset_index().to_csv(dir_tableau + '2016_china_rate.tsv', sep='\t', index=False)
rare_2016[['China', 'Total', 'china_rate',]].reset_index()

country,hts_code,China,Total,china_rate
0,2805.30.0005,43100.0,43400.0,99.308756
1,2805.30.0010,64700.0,85000.0,76.117647
2,2805.30.0020,4060.0,7170.0,56.624826
3,2805.30.0050,37500.0,43400.0,86.405530
4,2805.30.0090,139000.0,157000.0,88.535032
5,2846.10.0010,801000.0,1150000.0,69.652174
6,2846.10.0050,424000.0,1020000.0,41.568627
7,2846.90.2005,3320000.0,3330000.0,99.699700
8,2846.90.2015,4000.0,9340.0,42.826552
9,2846.90.2040,70700.0,79700.0,88.707654
